In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'C:\Users\faiza\Desktop\web-scraping-\final.xlsx')

In [2]:
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [3]:
#import nltk
#nltk.download('all')

In [4]:
df.head()

,topic,title,article,date,link
0,BUSINESS,"MAHB faces double whammy of interstate ban, Tu...",KUALA LUMPUR: Affin Hwang Capital expects Mala...,"May 4, 2021 @ 10:40am",https://www.nst.com.my/business/2021/05/687692...
1,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened easier agains...,"May 3, 2021 @ 10:09am",https://www.nst.com.my/business/2021/05/687398...
2,COLUMNISTS,Biden needs fresh thinking and more than one term,FORGET about the first 100 days. US President ...,"May 3, 2021 @ 12:10am",https://www.nst.com.my/opinion/columnists/2021...
3,SUNDAY VIBES,MONEY THOUGHTS: Ikigai — The uncommon key to c...,HAVE you noticed how life's getting tougher on...,"May 2, 2021 @ 8:30am",https://www.nst.com.my/lifestyle/sunday-vibes/...
4,BUSINESS,"Labuan FSA appoints new chairman, director gen...",KUALA LUMPUR: Labuan Financial Services Author...,"Apr 30, 2021 @ 4:15pm",https://www.nst.com.my/business/2021/04/686731...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    107 non-null    object
 1   title    107 non-null    object
 2   article  107 non-null    object
 3   date     107 non-null    object
 4   link     107 non-null    object
dtypes: object(5)
memory usage: 4.3+ KB


In [6]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [7]:
df['article_clean']=df['article'].apply(clean)

In [8]:
dictionary = corpora.Dictionary(df['article_clean'])
print(dictionary.num_nnz)

20580


In [9]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['article_clean'] ]
print(len(doc_term_matrix))

107


In [10]:
lda = gensim.models.ldamodel.LdaModel

In [11]:
num_topics= 14
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

Wall time: 5.6 s


In [12]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.023*"per" + 0.022*"cent" + 0.014*"said" + 0.013*"malaysia" + 0.009*"bond" + 0.009*"sukuk" + 0.008*"market" + 0.007*"government" + 0.006*"billion" + 0.006*"2020"'),
 (1,
  '0.009*"said" + 0.007*"police" + 0.006*"price" + 0.005*"the" + 0.005*"travel" + 0.005*"city" + 0.005*"osaka" + 0.005*"interstate" + 0.004*"dollar" + 0.004*"pandemic"'),
 (2,
  '0.011*"said" + 0.007*"bank" + 0.007*"also" + 0.006*"per" + 0.005*"bitcoin" + 0.005*"cent" + 0.005*"investor" + 0.005*"year" + 0.005*"would" + 0.005*"energy"'),
 (3,
  '0.013*"art" + 0.009*"digital" + 0.008*"cat" + 0.005*"christie" + 0.005*"artist" + 0.004*"token" + 0.004*"sale" + 0.004*"nftbased" + 0.004*"torres" + 0.004*"foundation"'),
 (4,
  '0.013*"said" + 0.009*"new" + 0.006*"vaccine" + 0.006*"also" + 0.005*"asset" + 0.005*"country" + 0.004*"malaysia" + 0.004*"investment" + 0.004*"sabah" + 0.003*"china"'),
 (5,
  '0.020*"cent" + 0.020*"per" + 0.015*"said" + 0.010*"bank" + 0.009*"fund" + 0.009*"billion" + 0.008*"million" + 0.008*"m

In [13]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [14]:
lda_corpus = ldamodel[doc_term_matrix]

In [15]:
[doc for doc in lda_corpus]

[[(0, 0.0003175488),
  (1, 0.00031754785),
  (2, 0.00031754829),
  (3, 0.00031754715),
  (4, 0.00031754797),
  (5, 0.0003175495),
  (6, 0.00031754846),
  (7, 0.9958719),
  (8, 0.0003175481),
  (9, 0.00031754942),
  (10, 0.00031754805),
  (11, 0.0003175486),
  (12, 0.00031754756),
  (13, 0.00031754872)],
 [(0, 0.00050680444),
  (1, 0.0005068034),
  (2, 0.0005068036),
  (3, 0.00050680287),
  (4, 0.0005068039),
  (5, 0.009778494),
  (6, 0.00050680415),
  (7, 0.0005068035),
  (8, 0.00050680456),
  (9, 0.7697377),
  (10, 0.21490896),
  (11, 0.00050680386),
  (12, 0.0005068032),
  (13, 0.00050680374)],
 [(0, 7.710308e-05),
  (1, 7.710288e-05),
  (2, 7.710311e-05),
  (3, 7.710294e-05),
  (4, 7.710313e-05),
  (5, 7.7103155e-05),
  (6, 7.710312e-05),
  (7, 7.710325e-05),
  (8, 0.13987137),
  (9, 0.8592034),
  (10, 7.710301e-05),
  (11, 7.7103105e-05),
  (12, 7.7102966e-05),
  (13, 7.71033e-05)],
 [(0, 0.00012876165),
  (1, 0.0001287616),
  (2, 0.00012876168),
  (3, 0.00012876146),
  (4, 0.00012

In [16]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.07142857106517427


In [17]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

In [18]:
df.iloc[cluster1]

,topic,title,article,date,link,article_clean
7,BUSINESS,Strong sukuk interest shows Malaysia's ability...,KUALA LUMPUR: The strong interest in its inter...,"Apr 27, 2021 @ 12:17pm",https://www.nst.com.my/business/2021/04/685792...,"[kuala, lumpur, strong, interest, internationa..."
12,BUSINESS,Malaysia's first sukuk in five years overwhelm...,KUALA LUMPUR: Malaysia's first global sukuk is...,"Apr 23, 2021 @ 10:01am",https://www.nst.com.my/business/2021/04/684652...,"[kuala, lumpur, malaysia, first, global, sukuk..."
18,BUSINESS,Malaysia bond laggards show signs of recovery,MALAYSIAN corporate dollar bonds are the worst...,"Apr 14, 2021 @ 4:05pm",https://www.nst.com.my/business/2021/04/682290...,"[malaysian, corporate, dollar, bond, worst, pe..."
28,BUSINESS,Ringgit slides against USD at opening amid saf...,KUALA LUMPUR: The ringgit slid slightly agains...,"Mar 26, 2021 @ 10:24am",https://www.nst.com.my/business/2021/03/677118...,"[kuala, lumpur, ringgit, slid, slightly, green..."
30,BUSINESS,Ringgit slips to 4.1330 versus USD at opening ...,KUALA LUMPUR: The ringgit slipped further vers...,"Mar 25, 2021 @ 10:05am",https://www.nst.com.my/business/2021/03/676818...,"[kuala, lumpur, ringgit, slipped, versus, u, d..."
63,BUSINESS,"Musk says he's a Bitcoin 'supporter,' buffetin...",Elon Musk once again sent ripples through the ...,"Feb 1, 2021 @ 5:16pm",https://www.nst.com.my/business/2021/02/662164...,"[elon, musk, sent, ripple, price, bitcoin, dec..."
65,BUSINESS,Azmin: Malaysia's exports rebounded in second ...,KUALA LUMPUR: Malaysia's external trade perfor...,"Jan 30, 2021 @ 11:27am",https://www.nst.com.my/business/2021/01/661700...,"[kuala, lumpur, malaysia, external, trade, per..."
93,BUSINESS,Improving sentiment boosts emerging East Asian...,KUALA LUMPUR: Accommodative monetary stance su...,"Nov 25, 2020 @ 11:41am",https://www.nst.com.my/business/2020/11/644138...,"[kuala, lumpur, accommodative, monetary, stanc..."
102,BUSINESS,RCEP signing helps boost ringgit on Monday,KUALA LUMPUR: The signing of Regional Comprehe...,"Nov 17, 2020 @ 11:16am",https://www.nst.com.my/business/2020/11/641752...,"[kuala, lumpur, signing, regional, comprehensi..."


In [19]:
df.iloc[cluster5]

,topic,title,article,date,link,article_clean
27,CRIME & COURTS,Hunt on for Bitcoin mining culprit behind TNB'...,MELAKA: Police here are hunting for a man beli...,"Mar 26, 2021 @ 7:39pm",https://www.nst.com.my/news/crime-courts/2021/...,"[melaka, police, hunting, man, believed, main,..."
41,WORLD,How vaccines became ammunition in global diplo...,PARIS: Covid-19 vaccines are not just coveted ...,"Mar 5, 2021 @ 11:38am",https://www.nst.com.my/world/world/2021/03/671...,"[paris, covid19, vaccine, coveted, protection,..."
50,BUSINESS,Ringgit opens lower against US dollar,KUALA LUMPUR: The ringgit extended its downtre...,"Feb 26, 2021 @ 9:41am",https://www.nst.com.my/business/2021/02/669153...,"[kuala, lumpur, ringgit, extended, downtrend, ..."
51,BUSINESS,Ringgit maintains 4.04 level against US dollar...,KUALA LUMPUR: After closing marginally higher ...,"Feb 25, 2021 @ 10:09am",https://www.nst.com.my/business/2021/02/668784...,"[kuala, lumpur, closing, marginally, higher, w..."
58,BUSINESS,RM marginally higher against USD on improving ...,KUALA LUMPUR: The ringgit opened marginally hi...,"Feb 4, 2021 @ 10:27am",https://www.nst.com.my/business/2021/02/662940...,"[kuala, lumpur, ringgit, opened, marginally, h..."
68,BUSINESS,Ringgit strengthens against all major currencies,KUALA LUMPUR: The ringgit reversed recent loss...,"Jan 26, 2021 @ 10:10am",https://www.nst.com.my/business/2021/01/660574...,"[kuala, lumpur, ringgit, reversed, recent, los..."
79,NATION,"Sabah can be a vibrant economic powerhouse, sa...",KOTA KINABALU: Sabah needs to position itself ...,"Jan 2, 2021 @ 7:57pm",https://www.nst.com.my/news/nation/2021/01/653...,"[kota, kinabalu, sabah, need, position, early,..."
86,SUNDAY VIBES,MONEY THOUGHTS: Use the buy-sell filter,IS there a simple way to figure out if we're i...,"Dec 27, 2020 @ 9:15am",https://www.nst.com.my/lifestyle/sunday-vibes/...,"[simple, way, figure, were, investing, specula..."
90,BUSINESS,APIB urges Chinese investors to look at Malays...,KUALA LUMPUR: Asia Pacific Investment Bank (AP...,"Nov 27, 2020 @ 1:46pm",https://www.nst.com.my/business/2020/11/644806...,"[kuala, lumpur, asia, pacific, investment, ban..."
94,BUSINESS,Ringgit opens lower against US dollar,KUALA LUMPUR: The ringgit retreated from yeste...,"Nov 24, 2020 @ 9:53am",https://www.nst.com.my/business/2020/11/643780...,"[kuala, lumpur, ringgit, retreated, yesterday,..."
